## Equinox

In [ ]:
!pip install equinox

In [ ]:
%reset -f

In [ ]:
import jax
import jax.numpy as jnp
import jax.random as jr
import equinox as eqx
from typing import Callable

### Imbriquer les modules d'equinox

In [ ]:
class Toto(eqx.Module):
    a: int
    cs:list[float]
    tens:jax.Array
    fn:Callable

    def __init__(self):
        self.a=3
        self.cs=[1.3,2,4]
        self.tens=jnp.ones([12])
        self.fn=lambda x:x**2

In [ ]:
Toto()

Un module qui en inclus un autre:

In [ ]:
class Bou(eqx.Module):
    b:float
    toto:Toto
    totos:list[Toto]

    def __init__(self):
        self.b=jnp.ones([2,2])
        self.toto=Toto()
        self.totos=[Toto() for _ in range(2)]

bou=Bou()
bou #<=> eqx.tree_pprint(bou)

Mais un module equinox, c'est aussi un simple pytree

In [ ]:
jax.tree.structure(bou)

In [ ]:
leaves=jax.tree.leaves(bou)

In [ ]:
#equinox est fort pour printer les pytrees
eqx.tree_pprint(leaves)

### Un modèle, c'est un pytree et une méthode `__call__`

In [ ]:
class Linear(eqx.Module):
    weight:jax.Array
    bias:jax.Array

    def __init__(self,n_in,n_out,*,key):
        self.weight=jax.random.normal(key,[n_in,n_out])*jnp.sqrt(2/n_in)
        self.bias = jnp.zeros([n_out])

    def __call__(self,X):
        return X@self.weight + self.bias

X=jnp.ones([3])
Linear(3,2,key=jax.random.key(0))(X)

In [ ]:
class NeuralNetwork(eqx.Module):
    layers: list
    multiplicative_bias: jax.Array
    name:str="NeuralNetwork"

    def __init__(self,n_in,n_out,n_hidden, key):
        key1, key2, key3 = jax.random.split(key, 3)
        self.layers = [Linear(n_in, n_hidden, key=key1),
                       jax.nn.relu,
                       Linear(n_hidden, n_hidden, key=key2),
                       jax.nn.relu,
                       Linear(n_hidden, n_out, key=key3)]

        self.multiplicative_bias = jnp.ones([n_out])


    def __call__(self, x):
        for layer in self.layers[:-1]:
            x = layer(x)

        return self.layers[-1](x) * self.multiplicative_bias


X=jnp.ones([3])
model=NeuralNetwork(3,2,8,key=jax.random.key(0))
model(X)

Ce modèle est donc un pytree où se mélange des tenseurs qui forment les paramètres, et des fonctions qui permettent de construire le `model_apply`.  

In [ ]:
for leaf in jax.tree.leaves(model):
    print(type(leaf))
    if hasattr(leaf,"shape"):
        print(leaf.shape)

Dans la syntaxe `model(X)` on a l'impression que l'on s'est éloigné de la phylosophie jax où la variable `params` apparait explicitement.

Mais en fait non, car `params` est juste un sous-ensemble des feuilles de `model`.

###  partition et combine sur un exemple simple

Il faut un moyen d'isoler les paramètres du modèle.

In [ ]:
pytree=[jnp.ones([5]),lambda x:x**2]

params,static=eqx.partition(pytree,eqx.is_array)

In [ ]:
params

In [ ]:
static

Combine pour mixer des pytree

In [ ]:
pytree1 = [None, 1, 2]
pytree2 = [0, None, None]
eqx.combine(pytree1, pytree2)

In [ ]:
pytree_back=eqx.combine(params,static)
pytree_back

### partition et combine sur un modèle

In [ ]:
model=NeuralNetwork(3,2,8,key=jax.random.key(0))
model(jnp.ones([3]))

In [ ]:
params,static=eqx.partition(model,eqx.is_array)

In [ ]:
params

In [ ]:
static

Créons un pytree avec la même structure que `params` mais avec que des zéros:

In [ ]:
params0=jax.tree.map(lambda a:jnp.zeros_like(a),params)

In [ ]:
model0=eqx.combine(params0,static)
model0(jnp.ones([3]))

## Revenir à la pattern `model_init, model_apply`

On peut utiliser equinox pour créer des modèles complexes, et ensuite vouloir travailler avec la pattern de base de jax:

* `model_init(rkey)` une fonction qui renvoie un paramètre aléatoire
* `model_apply(param,inp)` qui renvoie l'output.

In [ ]:
def model_init__model_apply__fnm(dim_in, dim_out,dim_hidden):


    def model_init(rkey):
        nn=NeuralNetwork(dim_in,dim_out,dim_hidden,key=rkey)
        params,_=
        return params


    #c'est mieux de créer le modèle equinox une seule fois, en dehors du model_apply
    #contrairement à model_init, la fonction model_apply sera appelée très souvent, il faut qu'elle soit très rapide
    nn=NeuralNetwork(dim_in,dim_out,dim_hidden,key=jr.key(42))
    _,static= eqx.partition(nn,eqx.is_array)
    @jax.jit
    def model_apply(param,inp):
        return ...

    return model_init,model_apply

In [ ]:
model_init,model_apply=model_init__model_apply__fnm(3,2,8)

In [ ]:
params= model_init(jax.random.key(0))

In [ ]:
X=jnp.ones([3])
model_apply(params,X)

In [ ]:
#--- To keep following outputs, do not run this cell! ---

Array([1.2503884, 1.0124156], dtype=float32)

***A vous:*** Comprenez vous pourquoi le résultat ci-dessus est nul ?

## Dériver  selon les `params` $

In [ ]:
model=NeuralNetwork(n_in=3,n_out=2,n_hidden=8,key=jr.key(0))

In [ ]:
params,static=eqx.partition(model, eqx.is_array)

In [ ]:
def loss_fn(params, static, xV, yV):
    # Recombinaison des paramètres et de la structure statique
    model = eqx.combine(params, static)
    ypredV = jax.vmap(model)(xV)
    return jnp.mean((ypredV - yV)**2)

In [ ]:
b=7
xV=jnp.ones([b,3])
yV=jnp.ones([b,2])

In [ ]:
loss, grads = jax.value_and_grad(loss_fn)(params, static, xV, yV)

In [ ]:
print(grads)

⇑ `grads` a la même structure que `params`

Effectuons une étape de gradient:

In [ ]:
lr=1e-2
params_updated=jax.tree.map(lambda pa,gr:pa-lr*gr,params,grads )

In [ ]:
model_updated=eqx.combine(params_updated,static)

### En plus court avec `eqx.filter_value_and_grad`

In [ ]:
model=NeuralNetwork(n_in=3,n_out=2,n_hidden=8,key=jr.key(0))

def compute_loss(model, xV, yV):
    preds = jax.vmap(model)(xV)  # Applique le modèle à chaque échantillon du batch
    return jnp.mean((preds-yV)**2)

b=7
xV=jnp.ones([b,3])
yV=jnp.ones([b,2])

loss, grads = eqx.filter_value_and_grad(compute_loss)(model, xV, yV)

grads


In [ ]:
lr=1e-2

def update(param_part,grad_part):
    if grad_part is None: #ce None correspond à une partie statique (donc filtrée) du modèle
        return param_part
    else:
        return param_part-lr*grad_part

model_updated = jax.tree.map(update,model,grads)

**Remarque:** Il y a plein de méthodes `eqx.filter_xxx` qui filtre avant de s'appliquer. Notamment `eqx.filter_jit`.

### Un exemple complet

In [ ]:
import optax


model=NeuralNetwork(n_in=3,n_out=2,n_hidden=8,key=jr.key(0))
optimizer = optax.adam(learning_rate=0.01)
opt_state = optimizer.init(eqx.filter(model, eqx.is_array))



@eqx.filter_jit
def compute_loss(model, xV, yV):
    preds = jax.vmap(model)(xV)  # Applique le modèle à chaque échantillon du batch
    return jnp.mean((preds-yV)**2)


@eqx.filter_jit
def update_step(model, opt_state, xV, yV):
    loss, grads = eqx.filter_value_and_grad(compute_loss)(model, xV, yV)
    updates, opt_state = optimizer.update(grads, opt_state, model)
    model = eqx.apply_updates(model, updates)
    return model, opt_state, loss


losses=[]
for step in range(100):
    # Générer des données synthétiques
    key = jr.key(0)
    xV = jax.random.normal(key, (100, 3))
    yV = jnp.zeros([100,2])
    # Effectuer une étape d'optimisation
    model, opt_state, loss = update_step(model, opt_state, xV, yV)
    losses.append(loss)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(losses);

***A vous:*** Observer le code précédent. Que fait `eqx.apply_update`. Réécriver cette fonction, testez.

## Petites choses $

### Inexact array

Qu'est-ce qu'un `is_inexact_array` ?

In [ ]:
for A in [jnp.array(1),jnp.array(1.),1.,object()]:
    print(eqx.is_array_like(A),eqx.is_array(A),eqx.is_inexact_array(A))

* `is_array_like` => tenseur ou scalaire
* `is_array` => tenseur
* `is_inexact_array` => tenseur de flotant ou de nombre complexe


Si, dans la création de nos modèles on utilise des tenseurs d'entier comme attribus, la fonction `eqx.is_array` va les laisser passer, et on aura des problèmes pour la dérivation. D'où l'intérêt de `eqx.is_inexact_array`.

### Sérialiser

Pas toujours facile de sérialiser les modèles. On peut utiliser `pickle` mais parfois cela bloque pour des raisons de sécurité informatqiue. Equinox propose une solution:

In [ ]:
import equinox as eqx
model_original = NeuralNetwork(3,2,8,key=jax.random.key(0))


def train(model):
    #ici on imagine plein de transformation des paramètres
    return model

model_trained = train(model_original)
eqx.tree_serialise_leaves("some_filename.eqx", model_trained)

In [ ]:
model_loaded = eqx.tree_deserialise_leaves("some_filename.eqx", model_original)

⇑ inconvénient: pour désérialiser, il faut fournir le `model_original`. En fait equinox ne sérialise que la partie "param", la partie statique étant récupérer dans `model_original`  

In [ ]:
X=jnp.ones([3])
model_loaded(X)